In [1]:
import pandas as pd
import os

In [2]:
period1_begining = pd.to_datetime('2021-01-15')
period1_end = pd.to_datetime('2021-06-30')
period2_begining = pd.to_datetime('2021-07-01')
period2_end = pd.to_datetime('2021-10-25')


def generate_periods(date):
      if period1_begining <= date.to_pydatetime() <= period1_end:
              return "Period 1"
      if period2_begining <= date.to_pydatetime() <= period2_end:
              return "Period 2"
      if date.to_pydatetime() > period2_end:
              return "Period 3"
      return "Exclude"

In [3]:
infected_vacc = pd.read_csv('https://data.egov.bg/resource/download/e9f795a8-0146-4cf0-9bd1-c0ba3d9aa124/csv')
hospitalized_vacc = pd.read_csv('https://data.egov.bg/resource/download/6fb4bfb1-f586-45af-8dd2-3385499c3664/csv')
deceased_vacc = pd.read_csv('https://data.egov.bg/resource/download/e6a72183-28e0-486a-b4e4-b5db8b60a900/csv')

In [4]:
infected_vacc.rename(columns={'Дата': 'date', 
                              'Ваксина':'vaccine', 
                              'Пол': 'gender', 
                              'Възрастова група': 'age_group', 
                              'Брой заразени': 'count_infected_vaccinated'}, inplace=True)

hospitalized_vacc.rename(columns={'Дата': 'date', 
                                  'Ваксина':'vaccine', 
                                  'Пол': 'gender', 
                                  'Възрастова група': 'age_group', 
                                  'Брой хоспитализирани': 'count_hospitalized_vaccinated'}, inplace=True)

deceased_vacc.rename(columns={'Дата': 'date', 
                              'Ваксина':'vaccine', 
                              'Пол': 'gender', 
                              'Възрастова група': 'age_group', 
                              'Брой починали': 'count_deceased_vaccinated'}, inplace=True)

In [5]:
# infected_vacc['date'] = pd.to_datetime(infected_vacc['date']) 
# hospitalized_vacc['date'] = pd.to_datetime(hospitalized_vacc['date']) 
# deceased_vacc['date'] = pd.to_datetime(deceased_vacc['date'])

In [6]:
for df in (infected_vacc, hospitalized_vacc, deceased_vacc):
    df['date'] = pd.to_datetime(df['date']) 
    df['Period'] = df.apply(lambda x: generate_periods(x['date']), axis=1)
    df.drop(['vaccine','gender', 'date'], axis=1, inplace=True)
    df = df.groupby(['Period', 'age_group'], as_index=False).sum()

In [7]:
# infected_vacc['Period'] = infected_vacc.apply(lambda x: generate_periods(x['date']), axis=1)
# hospitalized_vacc['Period'] = hospitalized_vacc.apply(lambda x: generate_periods(x['date']), axis=1)
# deceased_vacc['Period'] = deceased_vacc.apply(lambda x: generate_periods(x['date']), axis=1)

In [8]:
# infected_vacc.drop(['vaccine','gender', 'date'], axis=1, inplace=True)
# hospitalized_vacc.drop(['vaccine','gender', 'date'], axis=1, inplace=True)
# deceased_vacc.drop(['vaccine','gender', 'date'], axis=1, inplace=True)

In [9]:
# infected_vacc = infected_vacc.groupby(['Period', 'age_group'], as_index=False).sum()
# hospitalized_vacc = hospitalized_vacc.groupby(['Period', 'age_group'], as_index=False).sum()
# deceased_vacc = deceased_vacc.groupby(['Period', 'age_group'], as_index=False).sum()

In [10]:
mask = infected_vacc['age_group'] == '-'
infected_vacc = infected_vacc[~mask]
mask = hospitalized_vacc['age_group'] == '-'
hospitalized_vacc = hospitalized_vacc[~mask]
mask = deceased_vacc['age_group'] == '-'
deceased_vacc = deceased_vacc[~mask]

In [11]:
age_groups = ['12 - 14', '15 - 16', '17 - 19', '20 - 29', '30 - 39', 
              '40 - 49', '50 - 59', '60 - 69', '70 - 79', '80 - 89', '90+']

full_data = pd.DataFrame({
  'Period': [period for period in age_groups for period in ['Period 1', 'Period 2', 'Period 3']],
  'age_group': [age for age in age_groups for period in ['Period 1', 'Period 2', 'Period 3']]
})

In [ ]:
full_data = full_data.merge(infected_vacc, on=['Period', 'age_group'], how='left')
full_data = full_data.merge(hospitalized_vacc, on=['Period', 'age_group'], how='left')
full_data = full_data.merge(deceased_vacc, on=['Period', 'age_group'], how='left')

In [ ]:
full_data = full_data.sort_values(['Period', 'age_group'])

In [ ]:
full_data

In [ ]:
full_data.to_csv(os.path.join(os.getcwd(), "output_data", 'BG_Vaccinated_Infections_Periods[3].csv'), index=False)

In [15]:
full_data

,Period,age_group,count_infected_vaccinated,count_hospitalized_vaccinated,count_deceased_vaccinated
0,Period 1,12 - 14,NaN,NaN,NaN
3,Period 1,15 - 16,NaN,NaN,NaN
6,Period 1,17 - 19,NaN,NaN,NaN
9,Period 1,20 - 29,25.0,2.0,NaN
12,Period 1,30 - 39,58.0,2.0,NaN
15,Period 1,40 - 49,83.0,13.0,NaN
18,Period 1,50 - 59,132.0,17.0,NaN
21,Period 1,60 - 69,121.0,26.0,2.0
24,Period 1,70 - 79,57.0,15.0,2.0
27,Period 1,80 - 89,34.0,16.0,1.0
